# Scrape HTML Entity Page from Wikipedia

In [16]:
from bs4 import BeautifulSoup
import requests, re
import pandas as pd

In [17]:
URL = 'https://en.wikipedia.org/wiki/List_of_XML_and_HTML_character_entity_references'

WikiPedia is fussy about bots so we need to spoof headers.

In [18]:
headers = {
    'User-Agent': 'Lynx/2.8.6rel.5 libwww-FM/2.14'
}

In [19]:
with requests.get(URL, headers=headers) as r:
    soup = BeautifulSoup(r.text, 'html.parser')

There are two tables on the page. We want the second one.

In [20]:
html = soup.find_all("table", attrs = {'class': 'wikitable'})[1]

We could probably have used the `.jquery-tablesorter` selector as well.

Find the table dimensions.

In [21]:
rows = html.find_all("tr")
rows

[<tr style="background:#efefef">
 <th>Name</th>
 <th>Character</th>
 <th>Unicode code point (decimal)</th>
 <th>Standard</th>
 <th>DTD<sup class="reference" id="cite_ref-DTD_1-0"><a href="#cite_note-DTD-1">[a]</a></sup></th>
 <th>Old ISO subset<sup class="reference" id="cite_ref-ISOsubset_2-0"><a href="#cite_note-ISOsubset-2">[b]</a></sup></th>
 <th>Description<sup class="reference" id="cite_ref-Description_3-0"><a href="#cite_note-Description-3">[c]</a></sup></th>
 </tr>, <tr>
 <td>quot</td>
 <td>"</td>
 <td>U+0022 (34)</td>
 <td>HTML 2.0</td>
 <td>HTMLspecial</td>
 <td>ISOnum</td>
 <td><a href="/wiki/Quotation_mark" title="Quotation mark">quotation mark</a> <i>(APL quote)</i></td>
 </tr>, <tr>
 <td>amp</td>
 <td>&amp;</td>
 <td>U+0026 (38)</td>
 <td>Original html specification(html 1.0) and HTML 2.0</td>
 <td>HTMLspecial and <a class="external free" href="http://info.cern.ch/MarkUp/html-spec/html.dtd" rel="nofollow">http://info.cern.ch/MarkUp/html-spec/html.dtd</a> (originally)</td>


In [22]:
# Number of rows (subtracting one for headers).
#
nrows = len(rows) - 1

In [23]:
# Number of columns (from longest row).
#
row_lengths = [len(tr.find_all(['th', 'td'])) for tr in rows]
ncols = max(row_lengths)

In [24]:
'Table has %d rows and %d columns.' % (nrows, ncols)

'Table has 253 rows and 7 columns.'

In general tables can be rather complicated, with merged cells. You might need to look out for `colspan` and `rowspan` attributes. This table is nice and simple though.

## Write to CSV

In [25]:
col_index = [0, 1, 2, 6]

In [26]:
with open('wikipedia-html-entity.csv', 'w') as f:
    for tr in rows:
        # Find all header or data cells.
        cells = tr.find_all(['th', 'td'])
        # Select specific columns.
        cells = [cells[i] for i in col_index]
        f.write('|'.join([cell.text for cell in cells])+'\n')

**Note:** This CSV file will require some manual work because it has unmatched quote characters which play havoc with spreadsheets.

## Creating a Data Frame

Find the column headers. We'll obviously need to clean these up.

In [27]:
rows[0].find_all("th")

[<th>Name</th>,
 <th>Character</th>,
 <th>Unicode code point (decimal)</th>,
 <th>Standard</th>,
 <th>DTD<sup class="reference" id="cite_ref-DTD_1-0"><a href="#cite_note-DTD-1">[a]</a></sup></th>,
 <th>Old ISO subset<sup class="reference" id="cite_ref-ISOsubset_2-0"><a href="#cite_note-ISOsubset-2">[b]</a></sup></th>,
 <th>Description<sup class="reference" id="cite_ref-Description_3-0"><a href="#cite_note-Description-3">[c]</a></sup></th>]

Prepare the headers by converting to lowercase, removing footnotes and retaining only first word.

In [28]:
def prepare_header(th):
    th = th.text
    th = th.lower()
    th = re.sub('\[[^\]]*\]', '', th)
    th = th.split()[0]
    return(th)

column_names = [prepare_header(th) for th in rows[0].find_all("th")]

### Populating from a List

Populate a two dimensional list from the table cells.

In [29]:
table_data = [[td.text for td in tr.find_all('td')] for tr in rows[1:]]

Then create the data frame directly from the list.

In [30]:
df = pd.DataFrame(table_data, columns=table_names)

NameError: name 'table_names' is not defined

Again we select only a subset of columns.

In [ ]:
df = df[['name', 'character', 'unicode', 'description']]
df.head()

### Quick Manipulations on Data Frame

There's a method to write the entire data frame to a CSV file. This is a much more elegant approach to writing the CSV and immediately produces a valid file.

In [ ]:
df.to_csv('wikipedia-html-entity.csv', sep='|')

Extract the column names.

In [ ]:
df.columns

In [ ]:
df.dtypes

Check for missing values.

In [ ]:
df.isnull().sum()

Slice out a column.

In [ ]:
df['name']

In [35]:
%%bash
scrapy shell fetch('http://www.webberwentzel.com/wwb/content/en/ww/ww-our-people')


bash: line 1: syntax error near unexpected token `('
bash: line 1: `scrapy shell fetch('http://www.webberwentzel.com/wwb/content/en/ww/ww-our-people')'
